# DataFrames: Reading in messy data
     
In the [01-data-access](./01-data-access.ipynb) example we show how Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  One key difference, when using Dask Dataframes is that instead of opening a single file with a function like [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html), we typically open many files at once with [dask.dataframe.read_csv](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.read_csv). This enables us to treat a collection of files as a single dataset. Most of the time this works really well. But real data is messy and in this notebook we will explore a more advanced technique to bring messy datasets into a dask dataframe.

## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [1]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

<Client: 'inproc://10.1.0.4/8353/1' processes=1 threads=4, memory=2.00 GB>

## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [2]:
import dask
df = dask.datasets.timeseries()
df

Dask DataFrame Structure:
                   id    name        x        y
npartitions=30                                 
2000-01-01      int64  object  float64  float64
2000-01-02        ...     ...      ...      ...
...               ...     ...      ...      ...
2000-01-30        ...     ...      ...      ...
2000-01-31        ...     ...      ...      ...
Dask Name: make-timeseries, 30 tasks

In [3]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name, index=False);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [4]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [5]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

Dask DataFrame Structure:
                   id    name        x        y
npartitions=30                                 
                int64  object  float64  float64
                  ...     ...      ...      ...
...               ...     ...      ...      ...
                  ...     ...      ...      ...
                  ...     ...      ...      ...
Dask Name: read-csv, 30 tasks

In [6]:
df.head()

id    name         x         y
0  1020   Sarah -0.059290  0.592550
1   960  Ursula  0.588648 -0.727142
2  1004     Dan -0.451617  0.779843
3  1010     Tim  0.223168 -0.451790
4   977  Xavier  0.341697  0.332560

Let's look at some statistics on the data

In [7]:
df.describe().compute()

id             x             y
count  2.592000e+06  2.592000e+06  2.592000e+06
mean   9.999928e+02 -3.186987e-04 -4.600499e-04
std    3.163090e+01  5.772921e-01  5.774511e-01
min    8.520000e+02 -9.999984e-01 -9.999998e-01
25%    9.790000e+02 -4.953745e-01 -4.945869e-01
50%    1.000000e+03  6.240248e-03  5.149588e-03
75%    1.021000e+03  5.063429e-01  5.056337e-01
max    1.158000e+03  9.999989e-01  9.999983e-01

# Make some messy data

Now this works great, and in most cases dd.read_csv or dd.read_parquet etc are the preferred way to read in large collections of data files into a dask dataframe, but real world data is often very messy and some files may be broken or badly formatted. To simulate this we are going to create some fake messy data by tweaking our example csv files. For the file `data/2000-01-05.csv` we will replace with no data and for the file `data/2000-01-07.csv` we will remove the `y` column 

In [8]:
# corrupt the data in data/2000-01-05.csv
with open('data/2000-01-05.csv', 'w') as f:
    f.write('')

In [9]:
# remove y column from data/2000-01-07.csv
import pandas as pd
df = pd.read_csv('data/2000-01-07.csv')
del df['y']
df.to_csv('data/2000-01-07.csv', index=False)

In [10]:
!head data/2000-01-05.csv

In [11]:
!head data/2000-01-07.csv

id,name,x
987,Kevin,-0.382682374991111
989,Kevin,-0.7017349146448064
996,Dan,-0.9660769918509821
1007,Norbert,0.13630423297369054
1000,Dan,-0.22598874451907516
1056,Frank,-0.00634420874106767
979,Xavier,-0.09427530854841226
998,Bob,0.5172649173168621
1025,Wendy,0.00046158141237495615


# Reading the messy data

Let's try reading in the collection of files again

In [12]:
df = dd.read_csv('data/2000-*-*.csv')

In [13]:
df.head()

id    name         x         y
0  1020   Sarah -0.059290  0.592550
1   960  Ursula  0.588648 -0.727142
2  1004     Dan -0.451617  0.779843
3  1010     Tim  0.223168 -0.451790
4   977  Xavier  0.341697  0.332560

Ok this looks like it worked, let us calculate the dataset statistics again

In [14]:
df.describe().compute()

distributed.worker - WARNING -  Compute Failed
Function:  execute_task
args:      ((subgraph_callable, (<function pandas_read_text at 0x7f11781e9c10>, <function _make_parser_function.<locals>.parser_f at 0x7f117a33c1f0>, (<function read_block_from_file at 0x7f11711a5430>, <OpenFile '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-07.csv'>, 0, 64000000, b'\n'), b'id,name,x,y\n', {}, {'id': dtype('int64'), 'name': dtype('O'), 'x': dtype('float64'), 'y': dtype('float64')}, ['id', 'name', 'x', 'y'], False, False, None), 'read-csv-f09007e13be83eb633edc3077d412900'))
kwargs:    {}
Exception: ValueError('Length mismatch: Expected axis has 3 elements, new values have 4 elements')



ValueError: Length mismatch: Expected axis has 3 elements, new values have 4 elements

So what happened? 

When creating a dask dataframe from a collection of files, dd.read_csv samples the first few files in the dataset to determine the datatypes and columns available. Since it has not opened all the files it does not now if some of them are corrupt. Hence, `df.head()` works since it is only looking at the first file. `df.describe.compute()` fails because of the corrupt data in `data/2000-01-05.csv`

# Building a delayed reader

To get around this problem we are going to use a more advanced technique to build our dask dataframe. This method can also be used any time some custom logic is required when reading each file. Essentially, we are going to build a function that uses  pandas and some error checking and returns a pandas dataframe. If we find a bad data file we will either find a way to fix/clean the data or we will return and empty pandas dataframe with the same structure as the good data.

In [15]:
import numpy as np
import io

def read_data(filename):
    
    # for this to work we need to explicitly set the datatypes of our pandas dataframe 
    dtypes = {'id': int, 'name': str, 'x': float, 'y': float}
    try:
        # try reading in the data with pandas 
        df = pd.read_csv(filename, dtype=dtypes)
    except:
        # if this fails create an empty pandas dataframe with the same dtypes as the good data
        df = pd.read_csv(io.StringIO(''), names=dtypes.keys(), dtype=dtypes)
    
    # for the case with the missing column, add a column of data with NaN's
    if 'y' not in df.columns:
        df['y'] = np.NaN
        
    return df

Let's test this function on a good file and the two bad files

In [16]:
# test function on a normal file
read_data('data/2000-01-01.csv').head()

id    name         x         y
0  1020   Sarah -0.059290  0.592550
1   960  Ursula  0.588648 -0.727142
2  1004     Dan -0.451617  0.779843
3  1010     Tim  0.223168 -0.451790
4   977  Xavier  0.341697  0.332560

In [17]:
# test function on the empty file
read_data('data/2000-01-05.csv').head()

Empty DataFrame
Columns: [id, name, x, y]
Index: []

In [18]:
# test function on the file missing the y column
read_data('data/2000-01-07.csv').head()

id     name         x   y
0   987    Kevin -0.382682 NaN
1   989    Kevin -0.701735 NaN
2   996      Dan -0.966077 NaN
3  1007  Norbert  0.136304 NaN
4  1000      Dan -0.225989 NaN

# Assembling the dask dataframe

First we take our `read_data` function and convert it to a dask delayed function

In [19]:
from dask import delayed
read_data = delayed(read_data)

Let us look at what the function does now

In [20]:
df = read_data('data/2000-01-01.csv')
df

Delayed('read_data-2dbda7ac-e7de-4ede-a8d1-19988f23c944')

It creates a delayed object, to actually run read the file we need to run `.compute()`

In [21]:
df.compute()

id      name         x         y
0      1020     Sarah -0.059290  0.592550
1       960    Ursula  0.588648 -0.727142
2      1004       Dan -0.451617  0.779843
3      1010       Tim  0.223168 -0.451790
4       977    Xavier  0.341697  0.332560
...     ...       ...       ...       ...
86395  1001  Patricia -0.661209 -0.351411
86396   983     Laura -0.543706  0.343511
86397  1062     Alice  0.087097 -0.171182
86398   990    Hannah -0.386679  0.214413
86399  1001    Oliver -0.652539  0.998603

[86400 rows x 4 columns]

Now let's build a list of all the available csv files

In [22]:
# loop over all the files
from glob import glob
files = glob('data/2000-*-*.csv')
files

['data/2000-01-17.csv',
 'data/2000-01-10.csv',
 'data/2000-01-19.csv',
 'data/2000-01-06.csv',
 'data/2000-01-29.csv',
 'data/2000-01-27.csv',
 'data/2000-01-18.csv',
 'data/2000-01-08.csv',
 'data/2000-01-30.csv',
 'data/2000-01-02.csv',
 'data/2000-01-14.csv',
 'data/2000-01-01.csv',
 'data/2000-01-21.csv',
 'data/2000-01-13.csv',
 'data/2000-01-05.csv',
 'data/2000-01-12.csv',
 'data/2000-01-24.csv',
 'data/2000-01-25.csv',
 'data/2000-01-09.csv',
 'data/2000-01-23.csv',
 'data/2000-01-16.csv',
 'data/2000-01-15.csv',
 'data/2000-01-22.csv',
 'data/2000-01-11.csv',
 'data/2000-01-28.csv',
 'data/2000-01-03.csv',
 'data/2000-01-26.csv',
 'data/2000-01-07.csv',
 'data/2000-01-04.csv',
 'data/2000-01-20.csv']

Now we run the delayed read_data function on each file in the list

In [23]:
df = [read_data(file) for file in files]
df

[Delayed('read_data-0096a585-573d-4ef4-a01c-d308211c08a0'),
 Delayed('read_data-4aec897d-c639-4fa5-a897-05df48f990b6'),
 Delayed('read_data-dfa39884-d76b-4604-8db8-85044097f116'),
 Delayed('read_data-bb187423-0eb7-468c-ba12-45cfb24ed83f'),
 Delayed('read_data-4d41d087-eccd-475a-96d0-fc3dabcb36b1'),
 Delayed('read_data-f2461664-7e15-45d6-a21b-f5e77bea9a4b'),
 Delayed('read_data-1826b8d4-32fa-4a2d-8f43-e691f63c5cfa'),
 Delayed('read_data-8f7e6daa-b819-412b-85fb-ec466d07f5eb'),
 Delayed('read_data-46d33e22-ecdf-4b1d-b727-9c33dd9ea804'),
 Delayed('read_data-cdf56dba-ceff-43c7-b186-69e8b0ce4b9a'),
 Delayed('read_data-406de893-b8a7-4ed5-974e-97595aa744b7'),
 Delayed('read_data-24f2559f-8931-4278-87f7-5f84aa763d4e'),
 Delayed('read_data-b49f8c10-5b0a-4e81-838d-f33c4e9ccc1b'),
 Delayed('read_data-daf7e7e6-5dec-4189-bb57-66e4c3d36cc5'),
 Delayed('read_data-4d09f51f-8cb1-453a-9dd2-77488d702d3b'),
 Delayed('read_data-5b8dfbe8-a18e-4bd8-a873-d5754df4157c'),
 Delayed('read_data-2ae51758-d54c-4bdb-8

Then we use [dask.dataframe.from_delayed](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.from_delayed). This function creates a Dask DataFrame from a list of delayed objects as long as each delayed object returns a pandas dataframe. The structure of each individual dataframe returned must also be the same.

In [24]:
df = dd.from_delayed(df, meta={'id': int, 'name': str, 'x': float, 'y': float})
df

Dask DataFrame Structure:
                   id    name        x        y
npartitions=30                                 
                int64  object  float64  float64
                  ...     ...      ...      ...
...               ...     ...      ...      ...
                  ...     ...      ...      ...
                  ...     ...      ...      ...
Dask Name: from-delayed, 60 tasks

Note: we provided the dtypes in the `meta` keyword to explicitly tell Dask Dataframe what kind of dataframe to expect. If we did not do this Dask would infer this from the first delayed object which could be slow if it was a large csv file

## Now let's see if this works

In [25]:
df.head()

id    name         x         y
0   985   Kevin  0.736115  0.148173
1   977  Victor  0.596940 -0.175889
2   991   Sarah  0.873653 -0.474130
3   973   Edith  0.542031 -0.850698
4  1000   Alice -0.977006 -0.920772

In [26]:
df.describe().compute()

/usr/share/miniconda3/envs/dask-examples/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


id             x             y
count  2.505600e+06  2.505600e+06  2.419200e+06
mean   9.999927e+02 -3.801955e-04 -5.467497e-04
std    3.162811e+01  5.772895e-01  5.775135e-01
min    8.520000e+02 -9.999984e-01 -9.999998e-01
25%    9.790000e+02 -4.953745e-01 -4.945869e-01
50%    1.000000e+03  6.240248e-03  5.149588e-03
75%    1.021000e+03  5.063429e-01  5.056337e-01
max    1.158000e+03  9.999989e-01  9.999983e-01

## Success!

To recap, in this example, we looked at an approach to create a Dask Dataframe from a collection of many data files. Typically you would use built-in functions like `dd.read_csv` or `dd.read_parquet` to do this. Sometimes, this is not possible because of messy/corrupted files in your dataset or some custom processing that might need to be done. 

In these cases, you can build a Dask DataFrame with the following steps.

1. Create a regular python function that reads the data, performs any transformations, error checking etc and always returns a  Pandas dataframe with the same structure
2. Convert this read function to a delayed object using the `dask.delayed` function
3. Call each file in your dataset with the delayed data reader and assemble the output as a list of delayed objects
4. Used `dd.from_delayed` to covert the list of delayed objects to a Dask Dataframe 

This same technique can be used in other situations as well. Another example might be data files that require using a specialized reader, or several transformations before they can be converted to a pandas dataframe.